In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
admis = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv',
                    infer_datetime_format=True, parse_dates=[8,9],
                   dtype= {6:'str',12:'str',14:'str',15:'str'})
visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv',
                     infer_datetime_format=True, parse_dates=[12],
                    dtype={8:'str',9:'str',10:'str',11:'str'})
mem = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv',
                  infer_datetime_format=True, parse_dates=[5,61,62],
                 dtype= {6:'str',10:'str',11:'str',13:'str'})
print(admis.shape)
print(visits.shape)
print(mem.shape)

C:\Users\pfbab\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(6213, 23)
(231373, 18)
(61519, 63)


In [3]:
postvisit = 0
readmit = 0
#pull patient numbers that have more then one admission to loop through
for patnum in admis['Patient ID'].drop_duplicates():
    window = pd.to_datetime(0) #set window as a datetime to make first loop work
    #create dataframe for each patient number keeping only the final discharge per Start Date
    patient = admis[admis['Patient ID']==patnum]
    patient = patient.sort_values(['Encounter Start Date','Encounter End Date']).drop_duplicates(subset=['Patient ID','Encounter End Date'], keep='first')
    patient = patient.sort_values('Encounter End Date').drop_duplicates(subset=['Patient ID','Encounter Start Date'], keep='last')
    vpatient = visits[visits['Patient ID']==patnum].sort_values('Encounter Date')
    
    for rnum in range(0,len(patient)):
        if window < patient.iloc[rnum,8] :
            #test all admissions after rnum
            trigger = patient.iloc[rnum+1:,8]<=(patient.iloc[rnum,9] + pd.DateOffset(days=30)) #column 8=start, 9=end
            #add one if there are Trues
            readmit = readmit + int(trigger.sum() > 0)
            #save 30 day window date to remove dupicate dates
            window = (patient.iloc[rnum,9] + pd.DateOffset(days=30))
            if trigger.sum() > 0:
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = True
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'readmission'] = True
            else:
                admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = True
        else:
            admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'index_admis'] = False
        trigger7 = vpatient[vpatient['Encounter Date']>=patient.iloc[rnum,9]]['Encounter Date']  <=  (patient.iloc[rnum,9] + pd.DateOffset(days=7))
        postvisit += int(trigger7.sum() > 0)
        if trigger7.sum() > 0:
            admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'pcp_visit'] = True
        else:
            admis.loc[admis['Encounter ID']==patient.iloc[rnum,0],'pcp_visit'] = False


                


In [4]:
admis['index_admis'].value_counts()

True     5536
False     613
Name: index_admis, dtype: int64

In [5]:
(admis['index_admis'].isna()).sum()

64

In [6]:
len(admis)

6213

In [7]:
readmit

527

In [8]:
admis

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,index_admis,pcp_visit,readmission
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1417170523,1306890389,2013-01-12,2013-01-14,...,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,True,False,NaN
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1669599197,1902803315,2014-01-05,2014-01-12,...,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0,True,True,NaN
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1407880511,1902803315,2012-06-13,2012-06-21,...,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,True,True,NaN
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1407008907,1427055839,2012-12-06,2012-12-09,...,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,True,True,NaN
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1518045608,1992818256,2015-03-11,2015-03-14,...,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1720043417,1689772592,2015-02-06,2015-02-11,...,NaN,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,True,True,NaN
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1215968474,1427055839,2014-08-07,2014-08-10,...,664.11,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,True,True,NaN
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1861831836,1689608150,2015-11-21,2015-11-23,...,NaN,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,True,False,True
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1750544516,1689608150,2015-11-24,2015-11-27,...,NaN,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,False,False,NaN
